In [1]:
import re
import numpy as np
import pandas as pd
import tqdm

from sklearn.model_selection import train_test_split, StratifiedKFold

# torch
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset
import torchmetrics

# pytorch lightning
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

# transformers
from transformers import AdamW
from transformers.optimization import get_linear_schedule_with_warmup

# transformers 모델
from transformers import BertForSequenceClassification, BertTokenizer

In [2]:
df = pd.read_table('data/1. 실습용자료.txt', sep='|', encoding='cp949')
df.fillna('', inplace=True)
df['text'] = df['text_obj'] + ' ' + df['text_mthd'] + ' ' + df['text_deal']
# clean = re.compile("[^A-Za-zㄱ-힣 ]")
# df['text'] = df['text'].apply(lambda x: clean.sub(' ', str(x)))
df['target'] = df[['digit_1', 'digit_2', 'digit_3']].apply(lambda x: ' '.join(x.values.astype(str)), axis=1)

In [3]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣a-zA-Z{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x):
    x = pattern.sub(' ', x)
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

In [4]:
contents = df['text'].map(clean).to_list()

In [6]:
from kss import split_sentences

In [7]:
!mkdir ./test-mlm

In [8]:
!wget -O run_mlm.py https://raw.githubusercontent.com/huggingface/transformers/72aee83ced5f31302c5e331d896412737287f976/examples/pytorch/language-modeling/run_mlm.py

--2022-04-02 10:40:08--  https://raw.githubusercontent.com/huggingface/transformers/72aee83ced5f31302c5e331d896412737287f976/examples/pytorch/language-modeling/run_mlm.py
raw.githubusercontent.com (raw.githubusercontent.com)을(를) 해석하는 중... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
접속 raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... 접속됨.
HTTP 요청을 전송했습니다. 응답을 기다리는 중입니다... 200 OK
길이: 24078 (24K) [text/plain]
다음 위치에 저장: `run_mlm.py'

run_mlm.py          100%[===================>]  23.51K  --.-KB/s    / 0.007s   

2022-04-02 10:40:08 (3.47 MB/s) - `run_mlm.py' 저장됨 [24078/24078]



In [25]:
with open('korean_petitions_safe.txt', 'w') as f:
    for doc in tqdm.notebook.tqdm(contents):
        if doc:
            f.write(doc+'\n')
    f.close()

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [26]:
!head -n 10 korean_petitions_safe.txt

카센터에서 자동차부분정비 타이어오일교환
상점내에서 일반인을 대상으로 채소.과일판매
절단하여사업체에도매 공업용고무를가지고 합성고무도매
영업점에서 일반소비자에게 열쇠잠금장치
어린이집 보호자의 위탁을 받아 취학전아동보육
철 절삭.용접 카프라배관자재
음식점에서 접객시설을 갖추고 참치회(일본식)
쌀을 가지고 가공하여 떡제조
시청에서 재정과인력 일반공공행정 지방행정 집행
영업장에서 고객의뢰를 받아 내부전기공사


In [32]:
!python run_mlm.py \
    --model_name_or_path beomi/kcbert-base \
    --train_file korean_petitions_safe.txt \
    --do_train \
    --output_dir ./test-mlm

2022-04-02 10:59:14.821030: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
04/02/2022 10:59:15 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
04/02/2022 10:59:15 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_preci